# 1)

In [34]:
import numpy as np
from scipy.stats import bernoulli, multivariate_normal

# Given data points
data = np.array([
    [1, 0.6, 0.1],
    [0, -0.4, 0.8],
    [0, 0.2, 0.5],
    [1, 0.4, -0.1]
])

# Given parameters
pi = np.array([0.5, 0.5])
p = np.array([0.3, 0.7])
N1 = {
    'mean': np.array([1, 1]),
    'cov': np.array([[2, 0.5], [0.5, 2]])
}
N2 = {
    'mean': np.array([0, 0]),
    'cov': np.array([[1.5, 1], [1, 1.5]])
}

# Initialize arrays to store posterior probabilities
posterior_probs = np.zeros((len(data), 2))

# E-step: Compute posterior probabilities
for i in range(len(data)):
    print(f"\n\nX{ i }\n")
    likelihood_bernoulli_1 = p[0]**data[i, 0] * (1-p[0])**(1-data[i, 0])
    likelihood_bernoulli_2 = p[1]**data[i, 0] * (1-p[1])**(1-data[i, 0])
    print(f"Bernoulli Likelihood (Cluster 1): {likelihood_bernoulli_1}")
    print(f"Bernoulli Likelihood (Cluster 2): {likelihood_bernoulli_2}")
    
    likelihood_gaussian_1 = multivariate_normal.pdf(data[i, 1:], mean=N1['mean'], cov=N1['cov'])
    likelihood_gaussian_2 = multivariate_normal.pdf(data[i, 1:], mean=N2['mean'], cov=N2['cov'])
    print(f"Gaussian Likelihood (Cluster 1): {likelihood_gaussian_1}")
    print(f"Gaussian Likelihood (Cluster 2): {likelihood_gaussian_2}")
    
    likelihood_1 = likelihood_bernoulli_1 * likelihood_gaussian_1
    likelihood_2 = likelihood_bernoulli_2 * likelihood_gaussian_2
    print(f"likelihood (Cluster 1): {likelihood_1}")
    print(f"likelihood (Cluster 2): {likelihood_2}")
    
    denominator = pi[0] * likelihood_1 + pi[1] * likelihood_2
    print(f"Denominator): {denominator}")
    
    posterior_probs[i, 0] = (pi[0] * likelihood_bernoulli_1 * likelihood_gaussian_1) / denominator
    posterior_probs[i, 1] = (pi[1] * likelihood_bernoulli_2 * likelihood_gaussian_2) / denominator

print("Posterior Probabilities:\n", posterior_probs)



X0

Bernoulli Likelihood (Cluster 1): 0.3
Bernoulli Likelihood (Cluster 2): 0.7
Gaussian Likelihood (Cluster 1): 0.06657529920303393
Gaussian Likelihood (Cluster 2): 0.11961837142058572
likelihood (Cluster 1): 0.019972589760910178
likelihood (Cluster 2): 0.08373285999441
Denominator): 0.05185272487766009


X1

Bernoulli Likelihood (Cluster 1): 0.7
Bernoulli Likelihood (Cluster 2): 0.30000000000000004
Gaussian Likelihood (Cluster 1): 0.05004888824270901
Gaussian Likelihood (Cluster 2): 0.0681905803254947
likelihood (Cluster 1): 0.035034221769896304
likelihood (Cluster 2): 0.020457174097648412
Denominator): 0.027745697933772358


X2

Bernoulli Likelihood (Cluster 1): 0.7
Bernoulli Likelihood (Cluster 2): 0.30000000000000004
Gaussian Likelihood (Cluster 1): 0.06837452355368487
Gaussian Likelihood (Cluster 2): 0.12958103481626038
likelihood (Cluster 1): 0.047862166487579405
likelihood (Cluster 2): 0.038874310444878116
Denominator): 0.04336823846622876


X3

Bernoulli Likelihood (Cluster 

In [3]:
# M-step: Update model parameters
new_pi = np.mean(posterior_probs, axis=0)
print("PI:", new_pi)
print("\n\n")

# Update p
print("PRIORS\n")
new_p = np.zeros(2)
for j in range(2): # 2 clusters
    print("cluster:", j)
    p_sum = 0
    for i in range(len(data)):
        p_sum += data[i, 0] * posterior_probs[i, j]
    print("p_sum:", p_sum)
    denominator = np.sum(posterior_probs[:, j])
    print("denominator:", denominator)
    new_p[j] = p_sum / denominator
print("p:", new_p)
print("\n\n")

# Update N1 and N2
new_N1_mean = np.zeros(2)
new_N2_mean = np.zeros(2)
new_N1_cov = np.zeros((2, 2))
new_N2_cov = np.zeros((2, 2))

for j in range(2): # 2 clusters
    mean_sum = np.zeros(2)
    cov_sum = np.zeros((2, 2))
    for i in range(len(data)):
        mean_sum += posterior_probs[i, j] * data[i, 1:]
        cov_sum += posterior_probs[i, j] * np.outer(data[i, 1:] - [N1['mean'], N2['mean']][j], data[i, 1:] - [N1['mean'], N2['mean']][j])
    if j == 0:
        new_N1_mean = mean_sum / np.sum(posterior_probs[:, j])
        new_N1_cov = cov_sum / np.sum(posterior_probs[:, j])
    else:
        new_N2_mean = mean_sum / np.sum(posterior_probs[:, j])
        new_N2_cov = cov_sum / np.sum(posterior_probs[:, j])


# Update the model parameters
pi = new_pi
p = new_p

N1['mean'] = new_N1_mean
N2['mean'] = new_N2_mean
N1['cov'] = new_N1_cov
N2['cov'] = new_N2_cov


print("N1 mean:", N1['mean'])
print("N1 cov:\n", N1['cov'])
print("\n")
print("N2 mean:", N2['mean'])
print("N2 cov:\n", N2['cov'])


PI: [0.38616755 0.61383245]



PRIORS

cluster: 0
p_sum: 0.36151382107026986
denominator: 1.5446702187154808
cluster: 1
p_sum: 1.63848617892973
denominator: 2.455329781284519
p: [0.23403948 0.66731817]



N1 mean: [0.026509   0.50712978]
N1 cov:
 [[1.08904974 0.37439926]
 [0.37439926 0.33897366]]


N2 mean: [0.30914476 0.2104205 ]
N2 cov:
 [[ 0.20386353 -0.02360135]
 [-0.02360135  0.14840009]]


# 2)

In [35]:
# Given model parameters
π1 = 0.5
π2 = 0.5
p1 = 0.3
p2 = 0.7
μ1 = np.array([1, 1])
Σ1 = np.array([[2, 0.5], [0.5, 2]])
μ2 = np.array([0, 0])
Σ2 = np.array([[1.5, 1], [1, 1.5]])

# New observation
x_new = np.array([1, 0.3, 0.7])

# Calculate the likelihoods for the new observation
likelihood_cluster1 = (
    bernoulli.pmf(1, p1) *
    multivariate_normal.pdf(x_new[1:], mean=μ1, cov=Σ1)
)

likelihood_cluster2 = (
    bernoulli.pmf(1, p2) *
    multivariate_normal.pdf(x_new[1:], mean=μ2, cov=Σ2)
)

# Calculate unnormalized posteriors
unnormalized_posterior1 = π1 * likelihood_cluster1
unnormalized_posterior2 = π2 * likelihood_cluster2

# Normalize the posteriors
posterior1 = unnormalized_posterior1 / (unnormalized_posterior1 + unnormalized_posterior2)
posterior2 = unnormalized_posterior2 / (unnormalized_posterior1 + unnormalized_posterior2)

print("Posterior for Cluster 1:", posterior1)
print("Posterior for Cluster 2:", posterior2)

Posterior for Cluster 1: 0.20697271237150058
Posterior for Cluster 2: 0.7930272876284995


# 3)

In [33]:
# Given observations
observations = np.array([
    [1, 0.6, 0.1],
    [0, -0.4, 0.8],
    [0, 0.2, 0.5],
    [1, 0.4, -0.1]
])

# Given parameters
pi = np.array([0.5, 0.5])
p = np.array([0.3, 0.7])
mean = np.array([[1, 1], [0, 0]])
cov = np.array([[[2, 0.5], [0.5, 2]], [[1.5, 1], [1, 1.5]]])

# Step 1: Calculate the likelihood of each observation belonging to each cluster
likelihoods = np.zeros((len(observations), len(pi)))
for i in range(len(observations)):
    for j in range(len(pi)):
        likelihoods[i, j] = pi[j] * p[j] ** observations[i, 0] * (1 - p[j]) ** (1 - observations[i, 0]) * \
                            multivariate_normal.pdf(observations[i, 1:], mean[j], cov[j])
        
print(likelihoods)

# Step 2: Assign each observation to the cluster with the highest likelihood
assignments = np.argmax(likelihoods, axis=1)

print(assignments)

# Now, we have assigned each observation to a cluster (0 or 1).

# Step 3: Calculate the Manhattan distance between each observation and all other observations
manhattan_distances = np.zeros((len(observations), len(observations)))
for i in range(len(observations)):
    for j in range(len(observations)):
        manhattan_distances[i, j] = np.abs(observations[i] - observations[j]).sum()

print(manhattan_distances)


# Step 4: Calculate the average distance of each observation to all other observations in the same cluster
average_distances_same_cluster = np.zeros(len(observations))
for i in range(len(observations)):
    same_cluster_indices = np.where(assignments == assignments[i])[0]
    same_cluster_distances = manhattan_distances[i, same_cluster_indices]
    average_distances_same_cluster[i] = np.mean(same_cluster_distances)

print(average_distances_same_cluster)


# Step 5: Calculate the average distance of each observation to all observations in the other cluster
average_distances_other_cluster = np.zeros(len(observations))
for i in range(len(observations)):
    other_cluster_indices = np.where(assignments != assignments[i])[0]
    other_cluster_distances = manhattan_distances[i, other_cluster_indices]
    average_distances_other_cluster[i] = np.mean(other_cluster_distances)

# Step 6: Calculate the silhouette score for each observation
silhouette_scores = (average_distances_other_cluster - average_distances_same_cluster) / \
                    np.maximum(average_distances_other_cluster, average_distances_same_cluster)

# Step 7: Calculate the average silhouette score for both clusters
silhouette_cluster_0 = np.mean(silhouette_scores[assignments == 0])
silhouette_cluster_1 = np.mean(silhouette_scores[assignments == 1])

print("Silhouette score of the first cluster under Manhattan distance:", silhouette_cluster_0)
print("Silhouette score of the second cluster under Manhattan distance:", silhouette_cluster_1)



[[0.00998629 0.04186643]
 [0.01751711 0.01022859]
 [0.02393108 0.01943716]
 [0.00885705 0.04357503]]
[1 0 0 1]
[[0.  2.7 1.8 0.4]
 [2.7 0.  0.9 2.7]
 [1.8 0.9 0.  1.8]
 [0.4 2.7 1.8 0. ]]
[0.2  0.45 0.45 0.2 ]
Silhouette score of the first cluster under Manhattan distance: 0.7916666666666665
Silhouette score of the second cluster under Manhattan distance: 0.911111111111111
